In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib as mpl
import os
import numpy as np
from datetime import datetime, timedelta
import math
import orjson
from collections import Counter
from adjustText import adjust_text
import matplotlib.dates as mdates
import matplotlib.colors as mcolors
from concurrent.futures import ThreadPoolExecutor, as_completed

plt.rcParams["figure.figsize"] = [25, 10]

In [2]:
# Set list of test done
index_list_names = [
    "loadtest-webrtc-2024-kurento-2p",
    "loadtest-webrtc-2024-kurento-8p",
    "loadtest-webrtc-2024-kurento-3p-10s",
    "loadtest-webrtc-2024-kurento-3p-40s",
    "loadtest-webrtc-2024-pion-2p",
    "loadtest-webrtc-2024-pion-8p",
    "loadtest-webrtc-2024-pion-3p-10s",
    "loadtest-webrtc-2024-pion-3p-40s",
    "loadtest-webrtc-2024-mediasoup-2p",
    "loadtest-webrtc-2024-mediasoup-8p",
    "loadtest-webrtc-2024-mediasoup-3p-10s",
    "loadtest-webrtc-2024-mediasoup-3p-40s",
]

index_list_full_names = [
    "Kurento, 2 publishers per session",
    "Kurento, 8 publishers per session",
    "Kurento, 3 publishers and 10 subscribers per session",
    "Kurento, 3 publishers and 40 subscribers per session",
    "Pion, 2 publishers per session",
    "Pion, 8 publishers per session",
    "Pion, 3 publishers and 10 subscribers per session",
    "Pion, 3 publishers and 40 subscribers per session",
    "Mediasoup, 2 publishers per session",
    "Mediasoup, 8 publishers per session",
    "Mediasoup, 3 publishers and 10 subscribers per session",
    "Mediasoup, 3 publishers and 40 subscribers per session",
]

# index_list_names = [
#     "loadtest-webrtc-ov3-8p-pion",
#     "loadtest-webrtc-ov3-8p-mediasoup",
# ]

In [3]:
# Helper functions
def read_file(path):
    if os.path.exists(path):
        with open(path, 'r') as f:
            return orjson.loads(f.read())
    else:
        return []

def get_index_data(index_list_name, data_type):
    full_user_data = []

    for entry in os.scandir(f'stats/{index_list_name}'):
        if entry.is_dir():
            for sub_entry in os.scandir(entry.path):
                if sub_entry.is_dir():
                    data_path = os.path.join(sub_entry.path, data_type + '.json')
                    if os.path.exists(data_path):
                        data = read_file(data_path)
                        full_user_data.append({
                            'user': sub_entry.name,
                            'session': entry.name,
                            data_type: data
                        })
    return full_user_data


def is_publisher(index, user):
    return ("8p" in index) or (user <= 3)

def process_user_data(user_data):
    data = []
    for user in user_data:
        for event in user['events']:
            data.append({
                'user': int(user['user'].replace("User", "")),
                'session': int(user['session'].replace("LoadTestSession", "")),
                'event': event['event'] + (f"-{event['connection']}" if 'connection' in event else ""),
                'connection': event['connection'] if 'connection' in event else None,
                'timestamp': event['timestamp'],
            })
    return data

def process_index_list(index_list_name):
    user_data = get_index_data(index_list_name, "events")
    data = process_user_data(user_data)
    events_df = pd.DataFrame(data)
    events_df['timestamp'] = pd.to_datetime(events_df['timestamp'])
    events_df = events_df.sort_values(by='timestamp')
    return index_list_name, events_df


In [4]:
# Read all events files and save relevant data in DataFrames
events_dfs = {}

with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
    futures = {executor.submit(process_index_list, index_list_name): index_list_name for index_list_name in index_list_names}
    for future in as_completed(futures):
        index_list_name, events_df = future.result()
        events_dfs[index_list_name] = events_df

In [5]:
# Read log file and get specific lines with regex to get the connection failures
import re
start_end_times = {}
def process_log_file(index, log_file):
    regex_error = re.compile(r'(.*) ERROR .* Participant (.*) in session (.*) failed (\d+) times')
    regex_start = re.compile(r'(.*) INFO .* Starting test with .*')
    regex_end = re.compile(r'(.*) INFO .* Saved result in a .*')
    date_format = "%Y-%m-%d %H:%M:%S.%f %z"
    failed_attempts = []
    start_match = None
    end_match = None
    with open(log_file, 'r') as f:
        for line in f:
            if 'Participant' in line:
                error_match = regex_error.search(line)
                if error_match:
                    failed_attempts.append({
                        'user': error_match.group(2).replace('User', ''),
                        'session': error_match.group(3).replace('LoadTestSession', ''),
                        'attempts': int(error_match.group(4)),  # Ensure attempts is an integer
                        'timestamp': datetime.strptime(error_match.group(1) + " +0200", date_format),
                        'full_user': f"{error_match.group(2).replace('User', '')}-{error_match.group(3).replace('LoadTestSession', '')}",
                    })
            if not start_match:
                start_match = regex_start.search(line)
            if not end_match:
                end_match = regex_end.search(line)
    if start_match and end_match:
        start_end_times[index] = (datetime.strptime(start_match.group(1) + " +0200", date_format),
                                  datetime.strptime(end_match.group(1) + " +0200", date_format))
    return log_file, pd.DataFrame(failed_attempts)

failures = {}

with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
    futures = {executor.submit(process_log_file, index_list_name, f'logs/{index_list_name}.log'): index_list_name for index_list_name in index_list_names}
    for future in as_completed(futures):
        log_file, df = future.result()
        index_list_name = log_file.split('/')[-1].replace('.log', '')
        failures[index_list_name] = df

In [6]:
# For test purposes, paint all events in a timeline
# for test, events_df in events_dfs.items():
#     events_df = events_df.sort_values(by=["session", "user", "timestamp"])
#     fig, ax = plt.subplots()
#     event_types = events_df['event'].unique()
#     event_types.sort()
#     colors = mpl.colormaps.get_cmap('tab20')(np.linspace(0, 1, len(event_types)))
#     event_color = dict(zip(event_types, colors))
#     y_labels = []
#     y_ticks = 0
#     for session in events_dfs[test]['session'].unique():
#         events_df_session = events_df[events_df['session'] == session]
#         for user in events_df_session['user'].unique():
#             events_df_user = events_df_session[events_df_session['user'] == user]
#             y_ticks += 1
#             y_pos = [y_ticks] * len(events_df_user)
#             ax.scatter(events_df_user['timestamp'], y_pos, c=[event_color[event] for event in events_df_user['event']])
#             y_labels.append(f'{user} - {session}')

#     # add legend
#     legend_labels = []
#     for event, color in event_color.items():
#         legend_labels.append(plt.Line2D([0], [0], marker='o', color='w', label=event, markerfacecolor=color))
#     ax.legend(handles=legend_labels, loc='upper left')
#     fig.suptitle(f'{test}')
#     # y labels should be user - session
#     ax.set_yticks(range(1, y_ticks + 1))
#     ax.set_yticklabels(y_labels)


In [7]:
# For test purposes, paint all connection relevant events in a timeline
# for test in events_dfs.keys():
#     events_df = events_dfs[test]
#     events_df = events_df.sort_values(by=["session", "user", "timestamp"])
#     # events_df = events_df[
#     #     (events_df['event'] == 'connectionStart') |
#     #     # (events_df['event'] == 'signalConnected') |
#     #     # (events_df['event'] == 'connectedPublisher') |
#     #     ((events_df['event'] == 'streamCreated') & (events_df['connection'] == 'local')) |
#     #     ((events_df['event'] == 'streamDestroyed') & (events_df['connection'] == 'local')) |
#     #     (events_df['event'] == 'connectionEnd')
#     # ]
#     events_df = events_df[
#         ~(events_df['event'] == 'publisherStartSpeaking-local') & ~(events_df['event'] == 'publisherStopSpeaking-local') &
#         ~(events_df['event'] == 'accessAllowed-local')
#     ]
#     fig, ax = plt.subplots()
#     event_types = events_df['event'].unique()
#     event_types.sort()
#     colors = mpl.colormaps.get_cmap('tab20')(np.linspace(0, 1, len(event_types)))
#     event_color = dict(zip(event_types, colors))
#     y_labels = []
#     y_ticks = 0
#     for session in events_df['session'].unique():
#         events_df_session = events_df[events_df['session'] == session]
#         for user in events_df_session['user'].unique():
#             events_df_user = events_df_session[events_df_session['user'] == user]
#             y_ticks += 1
#             y_pos = [y_ticks] * len(events_df_user)
#             ax.scatter(events_df_user['timestamp'], y_pos, c=[event_color[event] for event in events_df_user['event']])
#             y_labels.append(f'{user} - {session}')


#     # add legend
#     legend_labels = []
#     for event, color in event_color.items():
#         legend_labels.append(plt.Line2D([0], [0], marker='o', color='w', label=event, markerfacecolor=color))
#     ax.legend(handles=legend_labels, loc='upper left')
#     fig.suptitle(f'{test}')
#     # y labels should be user - session
#     ax.set_yticks(range(1, y_ticks + 1))
#     ax.set_yticklabels(y_labels)

In [8]:
# Read all CPU data and save it in DataFrames
df_cpu_list = []
df_mem_list = []
for index in index_list_names:
    df_cpu = pd.read_csv(f"dfs_final/{index}-medianode.csv")
    df_mem = pd.read_csv(f"dfs_final/{index}-medianode.csv")
    df_cpu = df_cpu.drop(columns=["memory"]).dropna()
    df_mem = df_mem.drop(columns=["cpu"]).dropna()
    df_cpu["@timestamp"] = pd.to_datetime(df_cpu["@timestamp"], format="ISO8601")
    df_mem["@timestamp"] = pd.to_datetime(df_mem["@timestamp"], format="ISO8601")
    df_cpu_list.append(df_cpu)
    df_mem_list.append(df_mem)

In [9]:
# Read all QoE data and save it processed in DataFrames
index_qoe_df_list = []
for index in index_list_names:
    index_qoe_df = pd.DataFrame()
    for entry in os.scandir(f"qoe/{index}"):
        if entry.is_file():
            with open(f"qoe/{index}/{entry.name}", 'r') as f:
                qoe = orjson.loads(f.read())
            qoe_df = pd.DataFrame.from_dict(qoe)
            if not qoe_df.empty:
                qoe_df["vmaf"] = qoe_df["vmaf"] * 5
                qoe_df["visqol"] = qoe_df["visqol"] * 5
                info = entry.name.split("_")[0].split("-")
                session = info[1]
                userRecording = info[2]
                userBeingRecorded = info[3]
                qoe_df["userRecording"] = userRecording
                qoe_df["userBeingRecorded"] = userBeingRecorded
                qoe_df["session"] = session
                if index_qoe_df.empty:
                    index_qoe_df = qoe_df
                else:
                    index_qoe_df = pd.concat([index_qoe_df, qoe_df])
    index_qoe_df_list.append(index_qoe_df)


seconds_per_fragment = 17
seconds_per_padding = 2
seconds_per_video = seconds_per_fragment - seconds_per_padding
def set_qoe_times(df, user_1_start_connection_time, user_2_start_connection_time):
    cut_index_0_time = 5 # recording stats 5 seconds after connection is successful
    max_time = max(user_1_start_connection_time, user_2_start_connection_time)
    time_fn = lambda cut_index: max_time + timedelta(seconds=(cut_index * seconds_per_fragment) + cut_index_0_time).total_seconds()
    df.loc[:, "timestamp"] = df["cut_index"].apply(time_fn)


In [10]:
# Read batches data
with open(f"dfs_final/batches.json", 'r') as f:
    batches = orjson.loads(f.read())

In [ ]:
# Process events to get when users' connections started, when they connected to the session and when the connection failed, and how long it lasted
user_events_obj = {}

def get_connection_times(events):
    connection_times_recorded = []
    recorded_media_events = events[
        (events["event"] != "connectionStart") &
        (events["event"] != "connectionFail")
    ]
    for m, row in recorded_media_events.iterrows():
        timestamp = row["timestamp"]
        con_time = (timestamp - start_time).total_seconds()
        connection_times_recorded.append(con_time)
    return connection_times_recorded


def get_failure_times(events):
    connection_times_recorded = []
    recorded_media_events = events[
        events["event"] == "connectionFail"
    ]
    for m, row in recorded_media_events.iterrows():
        timestamp = row["timestamp"]
        con_time = (timestamp - start_time).total_seconds()
        connection_times_recorded.append(con_time)
    return connection_times_recorded

df_cpu_filtered_list = []
df_mem_filtered_list = []
for i, index_list_name in enumerate(index_list_names):
    #stats_df = get_index_data(index_list_name, "stats")
    events_df = events_dfs[index_list_name]
    events_df = events_df.sort_values(by=["session", "user", "timestamp"])
    start_time = events_df["timestamp"].min()
    end_time = events_df["timestamp"].max()
    test_duration = (end_time - start_time).total_seconds()
    cpu = df_cpu_list[i]
    mem = df_mem_list[i]
    cpu_filtered = cpu[(cpu["@timestamp"] >= start_time) & (cpu["@timestamp"] <= end_time)]
    cpu_filtered.loc[:, "@timestamp"] = cpu_filtered["@timestamp"].apply(lambda x: (x - start_time).total_seconds())
    cpu_filtered = cpu_filtered.sort_values(by="@timestamp")
    mem_filtered = mem[(mem["@timestamp"] >= start_time) & (mem["@timestamp"] <= end_time)]
    mem_filtered.loc[:, "@timestamp"] = mem_filtered["@timestamp"].apply(lambda x: (x - start_time).total_seconds())
    mem_filtered = mem_filtered.sort_values(by="@timestamp")
    df_cpu_filtered_list.append(cpu_filtered)
    df_mem_filtered_list.append(mem_filtered)
    index_qoe_df = index_qoe_df_list[i]
    sessions = events_df["session"].unique()
    user_events_obj[index_list_name] = {}
    prev_row_event = ""
    for session in sessions:
        users = events_df[events_df["session"] == session]["user"].unique()
        for user in users:
            full_user = f"{user}-{session}"
            user_events = events_df[
                (events_df["session"] == session) & (events_df["user"] == user)
            ]
            connection_starts = user_events[user_events["event"] == "connectionStart"]
            failures_user = failures[index_list_name][
                (failures[index_list_name]["full_user"] == full_user) &
                (failures[index_list_name]["timestamp"] >= start_time) &
                (failures[index_list_name]["timestamp"] <= end_time)
            ].copy()
            failures_user["event"] = "connectionFail"
            failures_user["recoverable"] = False
            if "kurento" in index_list_name or "mediasoup" in index_list_name:
                if is_publisher(index_list_name, user):
                    stream_created_locals = user_events[
                        user_events["event"] == "streamCreated-local"
                    ]
                    stream_playing_locals = user_events[
                        user_events["event"] == "streamPlaying-local"
                    ]
                else:
                    # we choose the first stream created as confirmation of media connection if the user is subscriber
                    stream_created_locals = user_events[
                        user_events["event"].str.startswith("streamCreated")
                    ]
                    stream_playing_locals = user_events[
                        user_events["event"].str.startswith("streamPlaying")
                    ]
                # combine connection starts and stream created locals and failures_user
                combined = pd.concat(
                    [
                        connection_starts,
                        stream_created_locals,
                        stream_playing_locals,
                        failures_user,
                    ]
                )
                # if there is more than one stream created and a stream playing, we only keep the first one (marks connection success)
                combined = combined.sort_values(by="timestamp")
                prev_row_event = ""
                for r, row in combined.iterrows():
                    is_prev_row_connected = prev_row_event.startswith("streamCreated") or prev_row_event.startswith("streamPlaying")
                    is_current_row_connected = row["event"].startswith("streamCreated") or row["event"].startswith("streamPlaying")
                    if is_prev_row_connected and is_current_row_connected:
                        combined = combined.drop(r)
                    prev_row_event = row["event"]
            else:
                reconnecting_events = user_events[user_events["event"].str.contains("Reconnecting")].copy()
                if not reconnecting_events.empty:
                    reconnecting_events["event"] = "connectionFail"
                    reconnecting_events["recoverable"] = True
                    failures_user = pd.concat([failures_user, reconnecting_events])
                # combine connection starts and stream created locals and failures_user
                if is_publisher(index_list_name, user):
                    stream_created_locals = user_events[
                        user_events["event"] == "LocalTrackPublished-local"
                    ]
                    stream_created_locals = stream_created_locals.sort_values(by="timestamp")
                    # The second one means the publisher has fully connected
                    stream_created_locals = stream_created_locals.iloc[::2, :]
                else:
                    # we choose the first stream created as confirmation of media connection if the user is subscriber
                    stream_created_locals = user_events[
                        user_events["event"].str.startswith("TrackSubscribed")
                    ]
                # combine connection starts and stream created locals and failures_user
                combined = pd.concat(
                    [connection_starts, stream_created_locals, failures_user]
                )
                combined = combined.sort_values(by="timestamp")
                for r, row in combined.iterrows():
                    is_prev_row_connected = prev_row_event.startswith("TrackSubscribed")
                    is_current_row_connected = row["event"].startswith("TrackSubscribed")
                    if is_prev_row_connected and is_current_row_connected:
                        combined = combined.drop(r)
                    prev_row_event = row["event"]
            if index_qoe_df.empty:
                index_qoe_df_filtered = pd.DataFrame()
            else:
                index_qoe_df_filtered = index_qoe_df[
                    (index_qoe_df["session"] == "LoadTestSession" + str(session)) &
                    (index_qoe_df["userRecording"] == "User" + str(user))
                ].copy()
                index_qoe_df_filtered = index_qoe_df_filtered.sort_values(by="cut_index")
            user_events_obj[index_list_name][full_user] = {}
            user_events_obj[index_list_name][full_user]["combined"] = combined
            user_events_obj[index_list_name][full_user]["qoe"] = index_qoe_df_filtered
            user_events_obj[index_list_name][full_user]["user"] = user
            user_events_obj[index_list_name][full_user]["session"] = session

In [12]:
# Calculate the intervals of time when a user is connected
def get_connection_time_intervals(user_events, start_time, test_duration):
    connection_time_intervals = {}
    users = user_events.keys()
    for username in users:
        group = user_events[username]['combined'].copy()
        group["timestamp"] = group["timestamp"].apply(lambda x: (x - start_time).total_seconds())
        connection_success_times = group.loc[~((group['event'] == 'connectionFail') | (group['event'] == 'connectionStart'))]['timestamp']
        connection_fail_times = group.loc[group['event'] == 'connectionFail']['timestamp']

        intervals = []
        for success_time in connection_success_times:
            fail_time = connection_fail_times[connection_fail_times > success_time].min()
            if pd.isnull(fail_time):
                intervals.append((success_time, test_duration))
            else:
                intervals.append((success_time, min(fail_time, test_duration)))

        connection_time_intervals[username] = intervals

    return connection_time_intervals

def intersect_intervals(intervals1, intervals2):
    intersection = []
    for interval1 in intervals1:
        for interval2 in intervals2:
            start = max(interval1[0], interval2[0])
            end = min(interval1[1], interval2[1])
            # there is a wait time of 5 seconds before starting recording
            start += 5
            if start < end:
                intersection.append((start, end))
    return intersection

intersections = {}
connection_time_intervals = {}
for i, index_list_name in enumerate(index_list_names):
    intersections[index_list_name] = {}
    start_time, end_time = start_end_times[index_list_name]
    cpu = df_cpu_list[i]
    last_cpu_timestamp = cpu.iloc[-1]["@timestamp"] + timedelta(seconds=10) # media server can be considered crashed if there aren't any more CPU data points 10 seconds past the last one
    end_time = min(end_time, last_cpu_timestamp)
    test_duration = (end_time - start_time).total_seconds()
    connection_time_intervals_index = get_connection_time_intervals(user_events_obj[index_list_name], start_time, test_duration)
    connection_time_intervals[index_list_name] = connection_time_intervals_index
    usernames = list(connection_time_intervals_index.keys())
    for i in range(len(usernames)):
        for j in range(i + 1, len(usernames)):
            _, session1 = usernames[i].split("-")
            _, session2 = usernames[j].split("-")
            if session1 == session2:
                user1_intervals = connection_time_intervals_index[usernames[i]]
                user2_intervals = connection_time_intervals_index[usernames[j]]
                intersection = intersect_intervals(user1_intervals, user2_intervals)
                if not usernames[i] in intersections[index_list_name]:
                    intersections[index_list_name][usernames[i]] = {}
                intersections[index_list_name][usernames[i]][usernames[j]] = intersection
                if not usernames[j] in intersections[index_list_name]:
                    intersections[index_list_name][usernames[j]] = {}
                intersections[index_list_name][usernames[j]][usernames[i]] = intersection



In [13]:
# Process QoE data to get the start and end of each fragment of time a QoE value is recorded
def set_frag_start_end(index_qoe_df, bi_intersections, fragment_duration, indexes):
    if len(bi_intersections) > 0:
        start = bi_intersections[0][0]
        end = bi_intersections[0][1]
        set_frag_start_end_aux(index_qoe_df, bi_intersections, fragment_duration, indexes, start, end, 0, 0)

def set_frag_start_end_aux(index_qoe_df, bi_intersections, fragment_duration, indexes, start, end, inters_j, index_i):
    if index_i < len(indexes):
        index = indexes[index_i]
        frag_end_time = start + fragment_duration
        if frag_end_time < end or math.isclose(frag_end_time, end, rel_tol=1e-9):
            index_qoe_df.loc[index, "start"] = start
            index_qoe_df.loc[index, "end"] = frag_end_time
            index_qoe_df.loc[index, "fragment_duration"] = fragment_duration
            start = frag_end_time
            set_frag_start_end_aux(index_qoe_df, bi_intersections, fragment_duration, indexes, start, end, inters_j, index_i + 1)
        else:
            j = inters_j + 1
            if j < len(bi_intersections):
                start = bi_intersections[j][0]
                end = bi_intersections[j][1]
                set_frag_start_end_aux(index_qoe_df, bi_intersections, fragment_duration, indexes, start, end, j, index_i)
            else:
                index_qoe_df.loc[index, "start"] = start
                index_qoe_df.loc[index, "end"] = end
                index_qoe_df.loc[index, "fragment_duration"] = end - start
                set_frag_start_end_aux(index_qoe_df, bi_intersections, fragment_duration, indexes, start, end, inters_j, index_i + 1)

for idx, index_list_name in enumerate(index_list_names):
    idx_intersections = intersections[index_list_name]
    index_qoe_df = index_qoe_df_list[idx]
    if not index_qoe_df.empty:
        index_qoe_df["session"] = index_qoe_df["session"].str.replace("LoadTestSession", "")
        index_qoe_df["userRecording"] = index_qoe_df["userRecording"].str.replace("User", "")
        index_qoe_df["userBeingRecorded"] = index_qoe_df["userBeingRecorded"].str.replace("User", "")
        index_qoe_df["full_user_recording"] = index_qoe_df["userRecording"] + "-" + index_qoe_df["session"]
        index_qoe_df["full_user_being_recorded"] = index_qoe_df["userBeingRecorded"] + "-" + index_qoe_df["session"]
        index_qoe_df = index_qoe_df.drop(columns=["userRecording", "userBeingRecorded", "session"]).reset_index(drop=True)
        # for user recording
        for full_user in idx_intersections.keys():
            qoe = index_qoe_df[index_qoe_df["full_user_recording"] == full_user]
            intersections_user = idx_intersections[full_user]
            # for user being recorded
            for full_user_being_recorded in intersections_user.keys():
                qoe_being_recorded = qoe.loc[index_qoe_df["full_user_being_recorded"] == full_user_being_recorded]
                bi_intersections = intersections_user[full_user_being_recorded]
                con_duration = 0
                for intersection in bi_intersections:
                    con_duration += intersection[1] - intersection[0]
                max_cut_index = qoe_being_recorded["cut_index"].max() + 1
                fragment_duration = con_duration / max_cut_index
                set_frag_start_end(index_qoe_df, bi_intersections, fragment_duration, qoe_being_recorded.index)
        index_qoe_df_list[idx] = index_qoe_df

In [14]:
# Helper function for getting the average QoE values for a specific user and in which times
def avg_qoe_intervals(index_qoe_df, metric, user):
    if index_qoe_df.empty:
        return []
    qoe = index_qoe_df[index_qoe_df["full_user_recording"] == user]
    qoe = qoe.sort_values(by=["start"])
    qoe["start"] = qoe["start"].round(decimals=5)
    qoe["end"] = qoe["end"].round(decimals=5)
    times = set()
    for index, row in qoe.iterrows():
        times.add(row["start"])
        times.add(row["end"])
    times = sorted(times)
    sub_intervals = []
    for i in range(len(times) - 1):
        start = times[i]
        end = times[i + 1]
        overlapping = []
        for r, row in qoe.iterrows():
            if (row["start"] < end and row["end"] > start):
                overlapping.append(row[metric])
        if len(overlapping) > 0:
            avg_qoe = sum(overlapping) / len(overlapping)
            sub_intervals.append(([start, end], avg_qoe))
    return sub_intervals

In [15]:
# Paint the connection progression of all users in a test and save the relevant data to DataFrames
plt.rcParams["figure.figsize"] = [40, 35]

colors_list = ["black", "orange", "yellow", "green", "darkblue"]
success_cmap = mcolors.LinearSegmentedColormap.from_list("success", colors_list)
normalize = mcolors.Normalize(vmin=0, vmax=5)

connection_stats_global = pd.DataFrame()

def get_con_type(next_color):
    if next_color == "magenta":
        return "connectionStart"
    elif next_color == "indigo":
        return "connectionSuccess"
    elif next_color == "red":
        return "connectionFail"
    else:
        return "QOE"

def calc_stats(cpu_filtered, prev_time, current_time, prev_attempt, n_batch, user, session, full_user, next_color, connection_stats, qoe=None, qoe_metric=None):
    total_duration = current_time - prev_time
    cpu_subset = cpu_filtered[(cpu_filtered['@timestamp'] >= prev_time) & (cpu_filtered['@timestamp'] <= current_time)]
    con_type = get_con_type(next_color)
    stats = {
        "type": con_type,
        "start_time": prev_time,
        "end_time": current_time,
        "total_duration": total_duration,
        "attempt": prev_attempt,
        "batch": n_batch,
        "user": user,
        "session": session,
        "full_user": full_user
    }
    if con_type == "QOE":
        stats["qoe"] = qoe
        stats["qoe_metric"] = qoe_metric
    # if there is no cpu data, add the next cpu data available
    if cpu_subset.empty:
        # seek closest cpu data to start_time or end_time
        cpu_prev_df = cpu_filtered[cpu_filtered['@timestamp'] < prev_time]
        cpu_next_df = cpu_filtered[cpu_filtered['@timestamp'] > current_time]
        # if one of the dfs is empty, we take the other
        if cpu_prev_df.empty:
            cpu_next = cpu_next_df.iloc[0]
            cpu_subset = cpu_filtered[cpu_filtered['@timestamp'] == cpu_next['@timestamp']]
        elif cpu_next_df.empty:
            cpu_prev = cpu_prev_df.iloc[-1]
            cpu_subset = cpu_filtered[cpu_filtered['@timestamp'] == cpu_prev['@timestamp']]
        else:
            cpu_next = cpu_next_df.iloc[0]
            cpu_prev = cpu_prev_df.iloc[-1]
            cpu_prev_diff = prev_time - cpu_prev['@timestamp']
            cpu_next_diff = cpu_next['@timestamp'] - current_time
            if cpu_prev_diff < cpu_next_diff:
                cpu_subset = cpu_filtered[cpu_filtered['@timestamp'] == cpu_prev['@timestamp']]
            else:
                cpu_subset = cpu_filtered[cpu_filtered['@timestamp'] == cpu_next['@timestamp']]
    stats["cpu_mean"] = cpu_subset['cpu'].mean()
    stats["cpu_start"] = cpu_subset['cpu'].iloc[0]
    stats["cpu_end"] = cpu_subset['cpu'].iloc[-1]
    stats["cpu_median"] = cpu_subset['cpu'].median()
    stats["cpu_max"] = cpu_subset['cpu'].max()
    stats["cpu_min"] = cpu_subset['cpu'].min()
    connection_stats.append(stats)
    return connection_stats

def paint_bar(ax, prev_time, current_time, y_ticks, prev_state, next_color):
    total_duration = current_time - prev_time
    yrange = (y_ticks - 0.5, 0.8)
    if prev_state != "connectionSuccess":
        segment = [(prev_time, total_duration)]
        ax.broken_barh(segment, yrange, facecolors=next_color)

def paint_success_bars(ax, segments, y_ticks):
    yrange = (y_ticks - 0.5, 0.8)
    batched_segments = []
    batched_colors = []

    for prev_time, current_time, qoe in segments:
        total_duration = current_time - prev_time
        segment = (prev_time, total_duration)
        color = success_cmap(normalize(qoe))

        batched_segments.append(segment)
        batched_colors.append(color)

    ax.broken_barh(batched_segments, yrange, facecolors=batched_colors)
    return batched_colors

def draw_cons_plot(i, index_list_name, metric):
    global connection_stats_global
    fig, ax = plt.subplots()
    y_labels = []
    y_ticks = 0
    user_count = 0
    connection_stats = []
    batch_info = batches[index_list_name]
    cpu_filtered = df_cpu_filtered_list[i]
    start_time, end_time = start_end_times[index_list_name]
    test_duration = (end_time - start_time).total_seconds()
    index_qoe_df = index_qoe_df_list[i]
    for full_user, full_user_obj in user_events_obj[index_list_name].items():
        user_count += 1
        if user_count <= batch_info["startingParticipants"]:
            n_batch = 0
        else:
            n_batch = math.floor((user_count - batch_info["startingParticipants"] - 1) / batch_info["batchSize"]) + 1
        user_split = full_user.split("-")
        user = user_split[0]
        session = user_split[1]
        y_labels.append(full_user)
        y_ticks += 1

        next_color = "white"
        prev_time = None
        prev_state = None
        prev_attempt = 0
        combined = full_user_obj["combined"]
        full_con_segments = connection_time_intervals[index_list_name][full_user]
        con_segments = avg_qoe_intervals(index_qoe_df, metric, full_user)
        # connection success
        for full_segment in full_con_segments:
            paint_bar(ax, full_segment[0], full_segment[1], y_ticks, "", "indigo")
            calc_stats(cpu_filtered, full_segment[0], full_segment[1], prev_attempt, n_batch, user, session, full_user, "indigo", connection_stats)
        segments_to_plot = [(segment[0][0], segment[0][1], segment[1]) for segment in con_segments]
        colors = paint_success_bars(ax, segments_to_plot, y_ticks)
        for segment, color in zip(con_segments, colors):
            calc_stats(cpu_filtered, segment[0][0], segment[0][1], prev_attempt, n_batch, user, session, full_user, color, connection_stats, segment[1], metric)
        # other connection events
        for r, row in combined.iterrows():
            current_time = (row["timestamp"] - start_time).total_seconds()
            if prev_time is not None:
                paint_bar(ax, prev_time, current_time, y_ticks, prev_state, next_color)
                calc_stats(cpu_filtered, prev_time, current_time, prev_attempt, n_batch, user, session, full_user, next_color, connection_stats)
            connectionSuccessful = (
                row["event"].startswith("streamCreated")
                or row["event"].startswith("streamPlaying")
                or row["event"].startswith("LocalTrackPublished")
                or row["event"].startswith("TrackSubscribed")
            )
            if row["event"] == "connectionStart":
                next_color = "magenta"
                prev_state = "connectionStart"
            elif connectionSuccessful:
                next_color = "indigo"
                prev_state = "connectionSuccess"
            else:
                next_color = "red"
                prev_attempt = row["attempts"]
                prev_state = "connectionFail"
            prev_time = current_time
        if prev_time is not None:
            paint_bar(ax, prev_time, test_duration, y_ticks, prev_state, next_color)
            calc_stats(cpu_filtered, prev_time, test_duration, prev_attempt, n_batch, user, session, full_user, next_color, connection_stats)
    connection_stats_df = pd.DataFrame(connection_stats)
    # add all rows to connection_stats_global
    if connection_stats_global.empty:
        connection_stats_global = connection_stats_df
        connection_stats_global["index"] = index_list_name
    else:
        connection_stats_df["index"] = index_list_name
        connection_stats_global = pd.concat([connection_stats_global, connection_stats_df])

    ax.set_yticks(range(1, y_ticks + 1))
    ax.set_yticklabels(y_labels)
    ax.set_title(f"Connection Progression ({metric}) - {index_list_full_names[i]}", pad=10)
    # y axis label
    ax.set_ylabel("User - Session")
    if user_count >= 150:
        # remove y labels
        ax.set_yticklabels([])
    # x axis label
    ax.set_xlabel("Time (s)")
    ax.set_xlim(-5, test_duration + 5)
    # grid
    ax.grid(True)
    # x axis ticks every 10 seconds
    if test_duration > 1000:
        ax.xaxis.set_major_locator(ticker.MultipleLocator(50))
    else:
        ax.xaxis.set_major_locator(ticker.MultipleLocator(10))
    # cpu in new axis
    ax2 = ax.twinx()
    ax2.set_ylabel("CPU (%)")
    # ax2 y axis should go from 0 to 1
    ax2.set_ylim(0, 1.01)
    ax2.set_yticks(np.arange(0, 1.01, 0.05))
    ax2.plot(cpu_filtered["@timestamp"], cpu_filtered["cpu"], color="black", marker="o", linewidth=4, markersize=10)
    # create legend
    legend_labels = [
        plt.Line2D([0], [0], color="magenta", lw=4, label="Connection Start"),
        plt.Line2D([0], [0], color="indigo", lw=4, label="Stream Created/Playing, No Video"),
        plt.Line2D([0], [0], color="darkblue", lw=4, label="Excellent QoE"),
        plt.Line2D([0], [0], color="green", lw=4, label="Good QoE"),
        plt.Line2D([0], [0], color="yellow", lw=4, label="Fair QoE"),
        plt.Line2D([0], [0], color="orange", lw=4, label="Poor QoE"),
        plt.Line2D([0], [0], color="black", lw=4, label="Bad QoE"),
        plt.Line2D([0], [0], color="red", lw=4, label="Failed Connection"),
        plt.Line2D([0], [0], color="black", lw=4, label="CPU", marker="o", markersize=10)
    ]

    ax.legend(handles=legend_labels, loc="upper left")
    fig.savefig(f"plots/qoe/connection_progression_{index_list_name[21:]}_{metric}.png", bbox_inches="tight")


In [ ]:
# Paint all tests
for i, index_list_name in enumerate(index_list_names):
    draw_cons_plot(i, index_list_name, "vmaf")
    draw_cons_plot(i, index_list_name, "visqol")

In [ ]:
# save connection relevant data to file for later reuse
os.makedirs("dfs_aux", exist_ok=True)
connection_stats_global.to_csv("dfs_aux/connection_stats.csv", index=False)

In [16]:
# alternative, read from file (avoids running long process)
connection_stats_global = pd.read_csv("dfs_aux/connection_stats.csv")

In [17]:
# Calculate how many publishers, subscribers, streams in and streams out there are at each time
def add_to(array, number, user, session, timestamp):
    array.append({
        "number": number,
        "user": user,
        "session": session,
        "timestamp": timestamp
    })
full_publishers_progression = {}
full_subscribers_progression = {}
full_streams_in_progression = {}
full_streams_out_progression = {}
def process_events_df(index_list_name, connection_stats):
    sessions = connection_stats['session'].unique()
    publishers_progression = []
    subscribers_progression = []
    streams_in_progression = []
    streams_out_progression = []

    previous_data = {'publishers': 0, 'subscribers': 0, 'streams_in': 0, 'streams_out': 0}
    for session in sessions:
        previous_data[session] = {'publishers': 0, 'subscribers': 0}

    for i, row in connection_stats.iterrows():
        session = row['session']
        user = row['user']
        type = row['type']
        full_user = row['full_user']
        timestamp = row['start_time']
        connected_users = set()

        if type == "connectionSuccess":
            if "0s" in index_list_name and int(user) > 3:
                subscribers = previous_data['subscribers'] + 1
                publishers_in_session = previous_data[session]['publishers']
                subscribers_in_session = previous_data[session]['subscribers'] + 1
                streams_out = 2 * publishers_in_session * (publishers_in_session - 1) + 2 * subscribers_in_session * publishers_in_session

                previous_data['subscribers'] = subscribers
                previous_data[session]['subscribers'] = subscribers_in_session
            else:
                publishers = previous_data['publishers'] + 1
                streams_in = previous_data['streams_in'] + 2
                publishers_in_session = previous_data[session]['publishers'] + 1
                subscribers_in_session = previous_data[session]['subscribers']
                streams_out = 2 * publishers_in_session * (publishers_in_session - 1) + 2 * subscribers_in_session * publishers_in_session

                previous_data['publishers'] = publishers
                previous_data['streams_in'] = streams_in
                previous_data[session]['publishers'] = publishers_in_session

            previous_data['streams_out'] = streams_out

            add_to(publishers_progression, previous_data['publishers'], user, session, timestamp)
            add_to(subscribers_progression, previous_data['subscribers'], user, session, timestamp)
            add_to(streams_in_progression, previous_data['streams_in'], user, session, timestamp)
            add_to(streams_out_progression, previous_data['streams_out'], user, session, timestamp)

            connected_users.add(full_user)

        elif type == "connectionFail" and full_user in connected_users:
            if "0s" in index_list_name and int(user) > 3:
                subscribers = previous_data['subscribers'] - 1
                publishers_in_session = previous_data[session]['publishers']
                subscribers_in_session = previous_data[session]['subscribers'] - 1
                streams_out = 2 * publishers_in_session * (publishers_in_session - 1) + 2 * subscribers_in_session * publishers_in_session

                previous_data['subscribers'] = subscribers
                previous_data[session]['subscribers'] = subscribers_in_session
            else:
                publishers = previous_data['publishers'] - 1
                streams_in = previous_data['streams_in'] - 2
                publishers_in_session = previous_data[session]['publishers'] - 1
                subscribers_in_session = previous_data[session]['subscribers']
                streams_out = 2 * publishers_in_session * (publishers_in_session - 1) + 2 * subscribers_in_session * publishers_in_session

                previous_data['publishers'] = publishers
                previous_data['streams_in'] = streams_in
                previous_data[session]['publishers'] = publishers_in_session

            previous_data['streams_out'] = streams_out

            add_to(publishers_progression, previous_data['publishers'], user, session, timestamp)
            add_to(subscribers_progression, previous_data['subscribers'], user, session, timestamp)
            add_to(streams_in_progression, previous_data['streams_in'], user, session, timestamp)
            add_to(streams_out_progression, previous_data['streams_out'], user, session, timestamp)

            connected_users.remove(full_user)

    publishers_progression = pd.DataFrame(publishers_progression)
    subscribers_progression = pd.DataFrame(subscribers_progression)
    streams_in_progression = pd.DataFrame(streams_in_progression)
    streams_out_progression = pd.DataFrame(streams_out_progression)

    return (index_list_name, publishers_progression, subscribers_progression, streams_in_progression, streams_out_progression)

# Use parallel processing to handle events dataframes
with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
    futures = [executor.submit(process_events_df, index_list_name, connection_stats_global[connection_stats_global["index"] == index_list_name]) for index_list_name in index_list_names]
    for future in as_completed(futures):
        (index_list_name, publishers_progression, subscribers_progression, streams_in_progression, streams_out_progression) = future.result()
        full_publishers_progression[index_list_name] = publishers_progression
        full_subscribers_progression[index_list_name] = subscribers_progression
        full_streams_in_progression[index_list_name] = streams_in_progression
        full_streams_out_progression[index_list_name] = streams_out_progression

In [21]:
# calculate certain values when in a CPU threshold interval
def calculate_connection_data(connection_type, connection_stats, metric_to_check, label, qoe_metric=None):
    connection_stats = connection_stats[connection_stats["type"] == connection_type].drop(columns=["type", "index", "user", "session"])
    connection_stats = connection_stats.sort_values(by="cpu_median").reset_index().drop(columns=["index"])

    cols_to_keep = ["full_user", "attempt", "batch", "total_duration", "cpu_median", "cpu_mean", "cpu_max", "cpu_min"]
    if connection_type == "QOE":
        cols_to_keep += ["qoe", "qoe_metric"]
    connection_stats = connection_stats[cols_to_keep]
    
    if connection_type == "QOE":
        connection_stats = connection_stats[connection_stats["qoe_metric"] == qoe_metric]

    thresholds = [
        { "threshold": "Full test", "condition": connection_stats["total_duration"] == connection_stats["total_duration"] },
        {"threshold": "Median CPU < 0.8", "condition": connection_stats["cpu_median"] < 0.8},
        {"threshold": "Median CPU < 0.9", "condition": connection_stats["cpu_median"] < 0.9},
        {"threshold": "Median CPU < 0.95", "condition": connection_stats["cpu_median"] < 0.95},
        {"threshold": "0.8 <= Median CPU < 0.9", "condition": (connection_stats["cpu_median"] >= 0.8) & (connection_stats["cpu_median"] < 0.9)},
        {"threshold": "0.9 <= Median CPU <= 1", "condition": (connection_stats["cpu_median"] >= 0.9)},
        {"threshold": "0.9 < Median CPU <= 0.95", "condition": (connection_stats["cpu_median"] >= 0.9) & (connection_stats["cpu_median"] < 0.95)},
        {"threshold": "0.95 < Median CPU", "condition": connection_stats["cpu_median"] >= 0.95},
    ]
    
    cpu_start_durations = []
    filtered_connection_stats = []
    
    for threshold in thresholds:
        filtered_stats = connection_stats[threshold["condition"]]
        mean = filtered_stats[metric_to_check].mean()
        median = filtered_stats[metric_to_check].median()
        min_val = filtered_stats[metric_to_check].min()
        max_val = filtered_stats[metric_to_check].max()
        std = filtered_stats[metric_to_check].std()
        
        cpu_start_durations.append({
            "threshold": threshold["threshold"],
            f"median {label}": median,
            f"mean {label}": mean,
            f"min {label}": min_val,
            f"max {label}": max_val,
            f"std {label}": std,
        })
        
        filtered_connection_stats.append(filtered_stats)
    
    cpu_start_durations_df = pd.DataFrame(cpu_start_durations)

    return (
        cpu_start_durations_df,
        *filtered_connection_stats,
    )

In [19]:
# Helper functions for comparing the certain values averages between different CPU thresholds using Mann-Whitney U test and t-test
from scipy import stats

def find_outliers_iqr(series):
    q1 = series.quantile(0.25)
    q3 = series.quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    outliers = series[(series < lower_bound) | (series > upper_bound)]
    return outliers

def get_cases(connection_stats, con_type="QOE", label="VMAF", qoe_metric="vmaf"):
    if con_type == "QOE":
        metric_to_check = "qoe"
    else:
        metric_to_check = "total_duration"
    cpu_start_durations_df, con_0_100, con_0_80, con_0_90, con_0_95, con_80_90, con_90_100, con_90_95, con_95_100 = calculate_connection_data(
        con_type, connection_stats, metric_to_check, label, qoe_metric
    )
    display(cpu_start_durations_df)
    cases = [
        # {
        #     "X": "CPU < 0.8",
        #     "Y": "CPU >= 0.8 and < 0.9",
        #     "len(X)": len(con_0_80),
        #     "len(Y)": len(con_80_90),
        #     "XSeries": con_0_80[metric_to_check],
        #     "YSeries": con_80_90[metric_to_check],
        # },
        # {
        #     "X": "CPU >= 0.8 and < 0.9",
        #     "Y": "CPU >= 0.9",
        #     "len(X)": len(con_80_90),
        #     "len(Y)": len(con_90_100),
        #     "XSeries": con_80_90[metric_to_check],
        #     "YSeries": con_90_100[metric_to_check],
        # },
        {
            "X": "CPU < 0.9",
            "Y": "CPU >= 0.9",
            "metric": label if con_type == "QOE" else "Total duration (s)",
            "len(X)": len(con_0_90),
            "len(Y)": len(con_90_100),
            "XSeries": con_0_90[metric_to_check],
            "YSeries": con_90_100[metric_to_check],
            "XSeriesCPU": con_0_90["cpu_median"],
            "YSeriesCPU": con_90_100["cpu_median"],
        },
        {
            "X": "CPU < 0.95",
            "Y": "CPU >= 0.95",
            "metric": label if con_type == "QOE" else "Total duration (s)",
            "len(X)": len(con_0_95),
            "len(Y)": len(con_95_100),
            "XSeries": con_0_95[metric_to_check],
            "YSeries": con_95_100[metric_to_check],
            "XSeriesCPU": con_0_95["cpu_median"],
            "YSeriesCPU": con_95_100["cpu_median"],
        },
    ]
    return cases

def clean_outliers(serie, outliers):
    serie = serie[~serie.isin(outliers)]
    return serie

def calculate_comparisons(X, Y, XSeries, YSeries, lenX, lenY, prev_80_data=None, prev_90_data=None, prev_95_data=None):
    ustat, p = stats.mannwhitneyu(XSeries, YSeries)
    tstat, tp = stats.ttest_ind(XSeries, YSeries)
    # wstat, wp, zw = stats.wilcoxon(XSeries, YSeries)
    shapiroX = stats.shapiro(XSeries)
    shapiroY = stats.shapiro(YSeries)
    brown_forsythe = stats.levene(XSeries, YSeries, center="median")
    fligner = stats.fligner(XSeries, YSeries, center="median")
    ks = stats.ks_2samp(XSeries, YSeries)
    ad = stats.anderson_ksamp([XSeries, YSeries])
    kw = stats.kruskal(XSeries, YSeries)

    meanX = XSeries.mean()
    meanY = YSeries.mean()
    medianX = XSeries.median()
    medianY = YSeries.median()
    stdX = XSeries.std()
    stdY = YSeries.std()
    pooled_std = np.sqrt(((lenX - 1) * stdX ** 2 + (lenY - 1) * stdY ** 2) / (lenX + lenY - 2))
    cohen_d = (meanX - meanY) / pooled_std
    rankBiserialCorrelation = (2 * ustat) / (lenX * lenY) - 1
    mean_diff = meanX - meanY
    median_diff = medianX - medianY
    std_diff = stdX - stdY
    mean_diff_percent = (mean_diff / meanY) * 100
    median_diff_percent = (median_diff / medianY) * 100
    std_diff_percent = (std_diff / stdY) * 100
    basics = {
        "X": X,
        "Y": Y,
        "len(X)": lenX,
        "len(Y)": lenY,
        "mean(X)": meanX,
        "mean(Y)": meanY,
        "median(X)": medianX,
        "median(Y)": medianY,
        "std(X)": stdX,
        "std(Y)": stdY,
        "mean diff": mean_diff,
        "median diff": median_diff,
        "std diff": std_diff,
        "mean diff %": mean_diff_percent,
        "median diff %": median_diff_percent,
        "std diff %": std_diff_percent,
    }
    comparisons = {
        "X": X,
        "Y": Y,
        "len(X)": lenX,
        "len(Y)": lenY,
        # "Wilcoxon (W)": wstat,
        # "p-value (Wilcoxon)": wp,
        # "z-statistic (Wilcoxon)": zw,
        "U (MWU)": ustat,
        "p-value (MWU)": p,
        "effect size (MWU, rank-biserial correlation)": rankBiserialCorrelation,
        "t (t-test)": tstat,
        "p-value (t-test)": tp,
        "effect size (t-test, Cohen's d)": cohen_d,
        "Shapiro X": shapiroX,
        "Shapiro Y": shapiroY,
        "Brown-Forsythe": brown_forsythe,
        "Fligner": fligner,
        "K-S": ks,
        "Anderson-Darling": ad,
        "Kruskal-Wallis": kw,
    }
    return basics, comparisons

def make_comparisons(cases, index=None, plots=True):
    comparisons = []
    comparisons_clean = []
    basics = []
    basics_clean = []
    if plots:
        figScatter, axsScatter = plt.subplots(1, 1, figsize=(30, 15))
        fig, axs = plt.subplots(1, len(cases)*2, figsize=(20, 8))
    prevFullSeries = None
    for i, case in enumerate(cases):
        X = case["X"]
        Y = case["Y"]
        lenX = case["len(X)"]
        lenY = case["len(Y)"]
        XSeries = case["XSeries"]
        YSeries = case["YSeries"]
        outliersX = find_outliers_iqr(XSeries)
        outliersY = find_outliers_iqr(YSeries)
        # clean outliers
        clean_XSeries = clean_outliers(XSeries, outliersX)
        clean_YSeries = clean_outliers(YSeries, outliersY)
        XSeriesCPU = case["XSeriesCPU"]
        YSeriesCPU = case["YSeriesCPU"]
        if plots:
            fullSeries = pd.concat([XSeries, YSeries])
            fullSeriesCPU = pd.concat([XSeriesCPU, YSeriesCPU])
            y_ticks = np.arange(0, 1.01, 0.05)
            x_ticks = y_ticks
            title = f"{index_list_full_names[index_list_names.index(index)] if index else 'All cases'} - CPU comparison"
            # plot scatter if not plotted previously over the same data
            if prevFullSeries is None or not prevFullSeries.equals(fullSeries):
                figScatter.set_tight_layout(True)
                figScatter.suptitle(f"{index if index else 'All cases'}")
                axsScatter.scatter(fullSeriesCPU, fullSeries)
                axsScatter.set_title(f"CPU vs {case['metric']}")
                axsScatter.set_xlabel("CPU")
                axsScatter.set_ylabel(case["metric"])
                axsScatter.set_xticks(x_ticks)
                axsScatter.set_yticks(y_ticks)
                axsScatter.grid(True)
                prevFullSeries = fullSeries
                figScatter.savefig(f"plots/scatters/{title if index else 'all'}.png")
            fig.set_tight_layout(True)
            fig.suptitle(f'{title if index else "All cases"}')
            # ax index
            idx = i * 2
            # plot boxplots
            axs[idx].boxplot([XSeries, YSeries], labels=[X + f"\n(n = {str(lenX)})", Y + f"\n(n = {str(lenY)})"])
            axs[idx].set_title(f"{X} vs {Y}")
            axs[idx].set_ylabel(case["metric"])
            axs[idx].set_ylim(0, 1)
            axs[idx].set_yticks(y_ticks)
            axs[idx].grid(True)
            # plot violin plots
            axs[idx+1].violinplot([XSeries, YSeries], showmeans=False, showmedians=True)
            axs[idx+1].set_title(f"{X} vs {Y}")
            axs[idx+1].set_ylabel(case["metric"])
            axs[idx+1].set_yticks(y_ticks)
            axs[idx+1].set_xticks([1, 2])
            axs[idx+1].set_xticklabels([X + f"\n(n = {str(lenX)})", Y + f"\n(n = {str(lenY)})"])
            axs[idx+1].grid(True)
            if i == len(cases) - 1:
                fig.savefig(f"plots/boxplots/{index if index else 'all'}.png")
        if lenX != 0 and lenY != 0:
            basic, comparison = calculate_comparisons(X, Y, XSeries, YSeries, lenX, lenY)
            basic["X"] = X
            basic["Y"] = Y
            basic["len(outliers(X))"] = len(outliersX)
            basic["len(outliers(Y))"] = len(outliersY)
            comparison["X"] = X
            comparison["Y"] = Y
            pearsonX = stats.pearsonr(XSeriesCPU, XSeries)
            pearsonY = stats.pearsonr(YSeriesCPU, YSeries)
            comparison["Pearson X"] = pearsonX
            comparison["Pearson Y"] = pearsonY
            comparisons.append(comparison)
            basics.append(basic)
            basic_clean, comparison_clean = calculate_comparisons(X, Y, clean_XSeries, clean_YSeries, lenX - len(outliersX), lenY - len(outliersY))
            basic_clean["X"] = X
            basic_clean["Y"] = Y
            basic_clean["len(outliers(X))"] = len(outliersX)
            basic_clean["len(outliers(Y))"] = len(outliersY)
            comparison_clean["X"] = X
            comparison_clean["Y"] = Y
            pearsonX_clean = stats.pearsonr(XSeriesCPU, clean_XSeries)
            pearsonY_clean = stats.pearsonr(YSeriesCPU, clean_YSeries)
            comparison_clean["Pearson X"] = pearsonX_clean
            comparison_clean["Pearson Y"] = pearsonY_clean
            comparisons_clean.append(comparison_clean)
            basics_clean.append(basic_clean)
    basics_df = pd.DataFrame(basics)
    basics_df.to_csv(f"tables/basics_{index if index else 'all'}.csv", index=False)
    comparisons_df = pd.DataFrame(comparisons)
    comparisons_df.to_csv(f"tables/comparisons_{index if index else 'all'}.csv", index=False)
    basics_clean_df = pd.DataFrame(basics_clean)
    basics_clean_df.to_csv(f"tables/basics_clean_{index if index else 'all'}.csv", index=False)
    comparisons_clean_df = pd.DataFrame(comparisons_clean)
    comparisons_clean_df.to_csv(f"tables/comparisons_clean_{index if index else 'all'}.csv", index=False)
    return basics_df, comparisons_df, basics_clean_df, comparisons_clean_df


In [ ]:
#Paint simple line chart with QoE values, averaging all users
for idx, index_list_name in enumerate(index_list_names):
    df_cpu = df_cpu_filtered_list[idx]
    df_mem = df_mem_filtered_list[idx]
    connection_stats = connection_stats_global[connection_stats_global["index"] == index_list_names[idx]]
    connection_stats["time_midpoint"] = (connection_stats["start_time"] + connection_stats["end_time"]) / 2
    if "kurento" in index_list_name:
        interval = 0.001
    else:
        interval = 1
    connection_stats["time_group"] = connection_stats["time_midpoint"] // interval * interval
    connection_stats_vmaf = connection_stats[connection_stats["qoe_metric"] == "vmaf"]
    connection_stats_vmaf = connection_stats_vmaf.groupby("time_group").agg({"qoe": "mean"}).reset_index()
    connection_stats_visqol = connection_stats[connection_stats["qoe_metric"] == "visqol"]
    connection_stats_visqol = connection_stats_visqol.groupby("time_group").agg({"qoe": "mean"}).reset_index()
    fig, ax1 = plt.subplots(figsize=(20, 6))
    ax1.plot(connection_stats_vmaf["time_group"], connection_stats_vmaf["qoe"], label="VMAF", color="blue")
    ax1.plot(connection_stats_visqol["time_group"], connection_stats_visqol["qoe"], label="VISQOL", color="red")
    ax1.set_xlabel("Time (s)")
    ax1.set_ylabel("QoE")
    ax1.set_title(f"QoE progression - {index_list_full_names[idx]}")
    ax1.grid()
    # y ranges from 0 to 1
    ax1.set_ylim(0, 1.01)

    ax2 = ax1.twinx()
    ax2.plot(df_cpu["@timestamp"], df_cpu["cpu"], color="black", marker="o", linewidth=1, markersize=3, label="CPU")
    ax2.set_ylabel("CPU (%)")
    # y axis should go from 0 to 1
    ax2.set_ylim(0, 1.01)

    ax3 = ax1.twinx()
    ax3.spines["right"].set_position(("axes", 1.05))
    ax3.spines["right"].set_visible(True)
    ax3.yaxis.set_label_position("right")
    ax3.yaxis.tick_right()
    ax3.plot(df_mem["@timestamp"], df_mem["memory"], color="green", marker="o", linewidth=1, markersize=3, label="Memory")
    ax3.set_ylabel("Memory (%)")
    # y axis should go from 0 to 1
    ax3.set_ylim(0, 1.01)

    fig.legend(loc="upper left")
    fig.savefig(f"plots/qoe/simple/qoe_progression_{index_list_name[21:]}.png", bbox_inches="tight")

In [ ]:
# Compare QoE values between different CPU thresholds
import warnings
# display("All cases together")
# cases = get_cases(connection_stats_global)
# comparisons = make_comparisons(cases)
# display(comparisons)
warnings.filterwarnings("ignore")
for index_list_name in index_list_names:
    connection_stats = connection_stats_global[
        connection_stats_global["index"] == index_list_name
    ].copy()
    display(index_list_name)
    cases = get_cases(connection_stats)
    basics, comparisons, basics_clean, comparisons_clean = make_comparisons(cases, index_list_name)
    display(basics, basics_clean, comparisons, comparisons_clean)
warnings.filterwarnings("default")

In [ ]:
warnings.filterwarnings("ignore")
for index_list_name in index_list_names:
    connection_stats = connection_stats_global[
        connection_stats_global["index"] == index_list_name
    ].copy()
    display(index_list_name)
    cases = get_cases(connection_stats, label="VISQOL", qoe_metric="visqol")
    basics, comparisons, basics_clean, comparisons_clean = make_comparisons(cases, index_list_name)
    display(basics, basics_clean, comparisons, comparisons_clean)
warnings.filterwarnings("default")

In [ ]:
# Compare connection start values when the connection start is followed by a connection success
import warnings
def remove_no_connections(connection_stats):
    # Identify connection starts that are not followed by a connection success
    mask = connection_stats["type"] == "connectionStart"
    
    # Collect indices of connection starts to be removed
    to_remove = []
    for i, row in connection_stats[mask].iterrows():
        # Check if there's a subsequent connectionSuccess for the same full_user and attempt
        next_row = connection_stats[
            (connection_stats["full_user"] == row["full_user"]) &
            (connection_stats["attempt"] == row["attempt"]) &
            (connection_stats["start_time"] > row["start_time"]) &
            (connection_stats["type"] == "connectionSuccess")
        ]
        
        # If no connectionSuccess found, mark for removal
        if next_row.empty:
            to_remove.append(i)
    
    # Remove the identified rows
    connection_stats.drop(to_remove, inplace=True)

    return connection_stats
# Connection start of successful connections
warnings.filterwarnings("ignore")
for index in index_list_names:
    connection_stats = connection_stats_global[connection_stats_global["index"] == index].copy()
    connection_stats = remove_no_connections(connection_stats)

    display(index)
    cases = get_cases(connection_stats, con_type="connectionStart", label="connection start duration (seconds)")
    basics, comparisons, basics_clean, comparisons_clean = make_comparisons(cases, index, plots=False)
    display(basics, basics_clean, comparisons, comparisons_clean)

warnings.filterwarnings("default")
# Connection start of successful connections
warnings.filterwarnings("ignore")
for index in index_list_names:
    connection_stats = connection_stats_global[connection_stats_global["index"] == index].copy()
    connection_stats = remove_no_connections(connection_stats)

    display(index)
    cases = get_cases(connection_stats, con_type="connectionStart", label="connection start duration (seconds)")
    basics, comparisons, basics_clean, comparisons_clean = make_comparisons(cases, index, plots=False)
    display(basics, basics_clean, comparisons, comparisons_clean)

warnings.filterwarnings("default")

In [ ]:
# For subscribers in tests with subscribers, compare connection start values when the connection start is followed by a connection success
def remove_no_connections_subs(connection_stats):
    con_stats = connection_stats.copy()
    # remove all connectionStart that are not followed by a connectionSuccess for that full_user and attempt
    for i, row in con_stats.iterrows():
        if row["type"] == "connectionStart":
            if int(row["user"]) > 3:
                con_stats = con_stats.drop(i)
            else:
                next_row = con_stats[
                    (con_stats["full_user"] == row["full_user"])
                    & (con_stats["attempt"] == row["attempt"])
                    & (con_stats["start_time"] > row["start_time"])
                ]
                if next_row.empty or next_row.iloc[0]["type"] != "connectionSuccess":
                    con_stats = con_stats.drop(i)
    return con_stats

warnings.filterwarnings("ignore")
connection_stats_subs = connection_stats_global[connection_stats_global["index"].str.contains("0s")].copy()
for index in index_list_names:
    if "0s" in index:
        connection_stats = connection_stats_subs[connection_stats_subs["index"] == index].copy()
        connection_stats = remove_no_connections_subs(connection_stats)
        display(index)
        cases = get_cases(connection_stats, con_type="connectionStart", label="connection start duration (seconds)")
        basics, comparisons, basics_clean, comparisons_clean = make_comparisons(cases, index, plots=False)
        display(basics, basics_clean, comparisons, comparisons_clean)
warnings.filterwarnings("default")


In [ ]:
# For subscribers in tests with subscribers, compare connection start values when the connection start is followed by a connection success
def remove_no_connections_subs(connection_stats):
    con_stats = connection_stats.copy()
    # remove all connectionStart that are not followed by a connectionSuccess for that full_user and attempt
    for i, row in con_stats.iterrows():
        if row["type"] == "connectionStart":
            if int(row["user"]) <= 3:
                con_stats = con_stats.drop(i)
            else:
                next_row = con_stats[
                    (con_stats["full_user"] == row["full_user"])
                    & (con_stats["attempt"] == row["attempt"])
                    & (con_stats["start_time"] > row["start_time"])
                ]
                if next_row.empty or next_row.iloc[0]["type"] != "connectionSuccess":
                    con_stats = con_stats.drop(i)
    return con_stats

warnings.filterwarnings("ignore")
connection_stats_subs = connection_stats_global[connection_stats_global["index"].str.contains("0s")].copy()
for index in index_list_names:
    if "0s" in index:
        connection_stats = connection_stats_subs[connection_stats_subs["index"] == index].copy()
        connection_stats = remove_no_connections_subs(connection_stats)
        display(index)
        cases = get_cases(connection_stats, con_type="connectionStart", label="connection start duration (seconds)")
        basics, comparisons, basics_clean, comparisons_clean = make_comparisons(cases, index, plots=False)
        display(basics, basics_clean, comparisons, comparisons_clean)
warnings.filterwarnings("default")


In [ ]:
# read all stats data and save to file (only if not already done)
# 3 types of stats
# webrtcStats: jitter etc.
# inbound: qoe calculated via inbound metrics
# outbound: qoe calculated via outbound metrics
# at the moment we only care about webrtcStats

def process_stats(index_list_name):
    stats = get_index_data(index_list_name, "stats")
    for stat in stats:
        if 'stats' in stat:
            user = stat['user']
            session = stat['session']
            in_stats = stat['stats']
            for s in in_stats:
                if 'webrtcStats' in s:
                    webrtc_stats = s['webrtcStats']
                    for ws in webrtc_stats:
                        if ('event' in ws) and (ws['event'] == 'stats'):
                            yield {
                                'user': user,
                                'session': session,
                                'data': ws
                            }

def add_user_data(data, user, session, remote_peer):
    for d in data:
        d['user'] = user
        d['session'] = session
        d['peerId'] = remote_peer
        # if cummulative metrics packetsLost and packetsSent/packetsReceived are present, calculate packetLoss
        if 'packetsLost' in d and ('packetsSent' in d or 'packetsReceived' in d):
            d['packetLoss'] = d['packetsLost'] / (d['packetsSent'] + d['packetsReceived'])


def separate_stats(index_list_name):

    stats_dict = {
        'inbound_audio': [],
        'inbound_video': [],
        'outbound_audio': [],
        'outbound_video': [],
        'remote_inbound_audio': [],
        'remote_inbound_video': [],
        'remote_outbound_audio': [],
        'remote_outbound_video': []
    }

    def process_data(data, user, session, peer_id, direction, type):
        if data:
            add_user_data(data, user, session, peer_id)
            stats_dict[f"{direction}_{type}"].extend(data)

    for full_data in process_stats(index_list_name):
        user = full_data['user']
        session = full_data['session']
        r = full_data['data']
        remotePeer = r['peerId']
        if 'data' in r:
            d = r['data']
            local_audio = d['audio']
            local_video = d['video']
            remote_audio = d['remote']['audio']
            remote_video = d['remote']['video']

            process_data(local_audio['inbound'], user, session, remotePeer, 'inbound', 'audio')
            process_data(local_audio['outbound'], user, session, remotePeer, 'outbound', 'audio')
            process_data(local_video['inbound'], user, session, remotePeer, 'inbound', 'video')
            process_data(local_video['outbound'], user, session, remotePeer, 'outbound', 'video')
            process_data(remote_audio['inbound'], user, session, remotePeer, 'remote_inbound', 'audio')
            process_data(remote_audio['outbound'], user, session, remotePeer, 'remote_outbound', 'audio')
            process_data(remote_video['inbound'], user, session, remotePeer, 'remote_inbound', 'video')
            process_data(remote_video['outbound'], user, session, remotePeer, 'remote_outbound', 'video')

    for key, value in stats_dict.items():
        if value:
            stats_dict[key] = pd.DataFrame(value)
            stats_dict[key].to_csv(f"dfs_stats_processed/{index_list_name}_{key}.csv", index=False)

for index_list_name in index_list_names:
    separate_stats(index_list_name)


In [ ]:
index = index_list_names[1]
direction = "remote_inbound"
type = "video"
metric = "bitrate"
strictness = 0


stats_inbound_video = pd.read_csv(f"dfs_stats_processed/{index}_{direction}_{type}.csv")
display(stats_inbound_video.columns)

In [ ]:
plt.rcParams["figure.figsize"] = [45, 30]

strictness = 0
directions = ["inbound", "outbound", "remote_inbound", "remote_outbound"]
types = ["audio", "video"]
jitter_threshold = 0.03

for type in types:
    for direction in directions:
        if direction == "inbound":
            metrics = [
                "bitrate",
                "packetRate",
                "jitter",
            ]
        elif direction == "outbound":
            metrics = [
                "bitrate",
                "packetRate",
            ]
        elif direction == "remote_inbound":
            metrics = [
                "jitter",
                "roundTripTime"
            ]
        elif direction == "remote_outbound":
            metrics = []
        for metric in metrics:
            fig, axs = plt.subplots(len(index_list_names), 1)
            for i, index in enumerate(index_list_names):
                filepath = f"dfs_stats_processed/{index}_{direction}_{type}.csv"
                if not os.path.exists(filepath):
                    continue
                stats_inbound_video = pd.read_csv(
                    f"dfs_stats_processed/{index}_{direction}_{type}.csv"
                )
                if not metric in stats_inbound_video.columns:
                    continue
                start_time = stats_inbound_video["timestamp"].min()
                end_time = stats_inbound_video["timestamp"].max()
                stats_inbound_video["timestamp"] = stats_inbound_video["timestamp"].apply(
                    lambda x: (x - start_time) / 1000
                )
                cpu = df_cpu_filtered_list[i]
                ax = axs[i]
                ax.set_title(f"{index} - {direction} {type} {metric}")
                ax.set_ylabel("CPU (%)")
                ax.set_xlabel("Time (s)")
                ax.set_ylim(0, 1.01)
                ax.plot(
                    cpu["@timestamp"],
                    cpu["cpu"],
                    color="black",
                    marker="o",
                    linewidth=4,
                    markersize=10,
                )
                ax2 = ax.twinx()
                ax2.set_ylabel(f"{metric}")
                # ax2.set_ylim(0, 100)
                # ax2.set_yticks(np.arange(0, 100, 5))
                stats_inbound_video["timestamp"] = stats_inbound_video["timestamp"].apply(
                    lambda x: round(x, strictness)
                )
                avg_jitter = (
                    stats_inbound_video[metric]
                    .groupby(stats_inbound_video["timestamp"])
                    .mean()
                )
                ax2.plot(
                    avg_jitter.index,
                    avg_jitter.values,
                    color="blue",
                    marker="o",
                )
                max_jitter = (
                    stats_inbound_video[metric].groupby(stats_inbound_video["timestamp"]).max()
                )
                min_jitter = (
                    stats_inbound_video[metric].groupby(stats_inbound_video["timestamp"]).min()
                )
                ax2.fill_between(
                    avg_jitter.index,
                    min_jitter.values,
                    max_jitter.values,
                    color="blue",
                    alpha=0.3,
                )
                # horizontal line on threshold
                if metric == "jitter":
                    ax2.axhline(y=jitter_threshold, color="red", linestyle="--")
                elif metric == "roundTripTime":
                    ax2.axhline(y=0.25, color="red", linestyle="--")
                ax.grid(True)
                # fig.savefig(f"plots/{index}_{direction}_{type}.png", bbox_inches="tight")

# TODO: Plot por usuario como las de QoE
# TODO: medias barra para QoS y media para QoE
# TODO: revisar secciones moradas de QoE -> son en los tests de pion 2p, cuando un solo usuario tiene una desconexión en la sesión el otro usuario ya no puede grabar (evento de trackunsubscribed)
# TODO: Lanzar los tests con mediasoup -> con OV2
# TODO: Bug a veces no se capturan las stats con una de las librerias -> Los payloads eran muy grandes en los tests de 8p con pion y mediasoup, arreglado

In [ ]:
# Paint the connection progression of all users in a test and save the relevant data to DataFrames
plt.rcParams["figure.figsize"] = [40, 35]
from decimal import Decimal, getcontext
getcontext().prec = 9

def avg_qos_intervals(stats, full_user, metric):
    time_interval = "1s"
    user_data = full_user.split("-")
    user = user_data[0]
    session = user_data[1]
    user_stats = stats[(stats["user"] == f"User{user}") & (stats["session"] == f"LoadTestSession{session}")]
    user_stats = user_stats.sort_values(by="timestamp").reset_index(drop=True)
    user_stats[metric] = user_stats[metric].apply(Decimal)
    if "bitrate" in metric or "packetRate" in metric:
        user_stats = user_stats[user_stats[metric] > 0]
    user_stats['timedelta'] = pd.to_timedelta(user_stats['timestamp'], unit='S')
    user_stats.set_index('timedelta', inplace=True)
    user_stats = user_stats.resample(time_interval).agg({metric: 'mean'}).dropna()
    user_stats['timestamp'] = user_stats.index.total_seconds().astype(int)
    user_stats['next_timestamp'] = user_stats['timestamp'].shift(-1)
    user_stats = user_stats.dropna(subset=['next_timestamp'])
    sub_intervals = list(zip(user_stats[['timestamp', 'next_timestamp']].values.tolist(), user_stats[metric]))
    return sub_intervals

def get_con_type(next_color):
    if next_color == "magenta":
        return "connectionStart"
    elif next_color == "indigo":
        return "connectionSuccess"
    elif next_color == "red":
        return "connectionFail"
    else:
        return "QOE"

def paint_bar_qos(ax, prev_time, current_time, y_ticks, prev_state, next_color):
    total_duration = current_time - prev_time
    yrange = (y_ticks - 0.5, 0.8)
    if prev_state != "connectionSuccess":
        segment = [(prev_time, total_duration)]
        ax.broken_barh(segment, yrange, facecolors=next_color)

def paint_success_bar_qos(ax, segments, y_ticks, success_cmap, normalize):
    yrange = (y_ticks - 0.5, 0.8)
    batched_segments = []
    batched_colors = []

    for prev_time, current_time, qos in segments:
        total_duration = current_time - prev_time
        segment = (prev_time, total_duration)
        color = success_cmap(normalize(qos))

        batched_segments.append(segment)
        batched_colors.append(color)
    
    ax.broken_barh(batched_segments, yrange, facecolors=batched_colors)

def draw_cons_plot_qos(i, index_list_name, direction, track_type, metric, vmin, vmax, colors_list):
    success_cmap = mcolors.LinearSegmentedColormap.from_list("success", colors_list)
    normalize = mcolors.Normalize(vmin, vmax)

    fig, ax = plt.subplots()
    y_labels = []
    y_ticks = 0
    user_count = 0
    cpu_filtered = df_cpu_filtered_list[i]
    start_time, end_time = start_end_times[index_list_name]
    test_duration = (end_time - start_time).total_seconds()
    stats = pd.read_csv(f"dfs_stats_processed/{index_list_name}_{direction}_{track_type}.csv")
    stats["timestamp"] = pd.to_datetime(stats["timestamp"], unit="ms") \
        .dt.tz_localize("UTC") \
        .apply(lambda x: (x - start_time).total_seconds())
    plt.ioff()
    for full_user, full_user_obj in user_events_obj[index_list_name].items():
        # display(f"User {full_user} started at time {datetime.now()}")
        if (("remote_inbound" in direction) or ("outbound" in direction)) and (not "8p" in index_list_name) and (int(full_user.split("-")[0]) > 3):
            continue
        user_count += 1
        y_labels.append(full_user)
        y_ticks += 1
        next_color = "white"
        prev_time = None
        prev_state = None
        combined = full_user_obj["combined"]
        full_con_segments = connection_time_intervals[index_list_name][full_user]
        con_segments = avg_qos_intervals(stats, full_user, metric)
        # connection success
        for full_segment in full_con_segments:
            paint_bar_qos(ax, full_segment[0], full_segment[1], y_ticks, "", "indigo")
        segments_to_plot = [(segment[0][0], segment[0][1], segment[1]) for segment in con_segments]
        paint_success_bar_qos(ax, segments_to_plot, y_ticks, success_cmap, normalize)
        # other connection events
        for r, row in combined.iterrows():
            current_time = (row["timestamp"] - start_time).total_seconds()
            if prev_time is not None:
                paint_bar_qos(ax, prev_time, current_time, y_ticks, prev_state, next_color)
            connectionSuccessful = (
                row["event"].startswith("streamCreated")
                or row["event"].startswith("streamPlaying")
                or row["event"].startswith("LocalTrackPublished")
                or row["event"].startswith("TrackSubscribed")
            )
            if row["event"] == "connectionStart":
                next_color = "magenta"
                prev_state = "connectionStart"
            elif connectionSuccessful:
                next_color = "indigo"
                prev_state = "connectionSuccess"
            else:
                next_color = "red"
                prev_state = "connectionFail"
            prev_time = current_time
        if prev_time is not None:
            paint_bar(ax, prev_time, test_duration, y_ticks, prev_state, next_color)
        # display(f"User {full_user} finished at time {datetime.now()}")

    ax.set_yticks(range(1, y_ticks + 1))
    ax.set_yticklabels(y_labels)
    ax.set_title(f"Connection Progression ({direction} {track_type} {metric}) - {index_list_full_names[i]}", pad=10)
    # y axis label
    ax.set_ylabel("User - Session")
    if user_count >= 150:
        # remove y labels
        ax.set_yticklabels([])
    # x axis label
    ax.set_xlabel("Time (s)")
    ax.set_xlim(-5, test_duration + 5)
    # grid
    ax.grid(True)
    # x axis ticks every 10 seconds
    ax.xaxis.set_major_locator(ticker.MultipleLocator(10))
    # cpu in new axis
    ax2 = ax.twinx()
    ax2.set_ylabel("CPU (%)")
    # ax2 y axis should go from 0 to 1
    ax2.set_ylim(0, 1.01)
    ax2.set_yticks(np.arange(0, 1.01, 0.05))
    ax2.plot(cpu_filtered["@timestamp"], cpu_filtered["cpu"], color="black", marker="o", linewidth=4, markersize=10)
    # create legend
    legend_labels = [
        plt.Line2D([0], [0], color="magenta", lw=4, label="Connection Start"),
        plt.Line2D([0], [0], color="indigo", lw=4, label="Stream Created/Playing, No stats recorded"),
        plt.Line2D([0], [0], color="darkblue", lw=4, label="Excellent QoS"),
        plt.Line2D([0], [0], color="green", lw=4, label="Good QoS"),
        plt.Line2D([0], [0], color="yellow", lw=4, label="Fair QoS"),
        plt.Line2D([0], [0], color="orange", lw=4, label="Poor QoS"),
        plt.Line2D([0], [0], color="black", lw=4, label="Bad QoS"),
        plt.Line2D([0], [0], color="red", lw=4, label="Failed Connection"),
        plt.Line2D([0], [0], color="black", lw=4, label="CPU", marker="o", markersize=10)
    ]

    ax.legend(handles=legend_labels, loc="upper left")
    plt.ion()
    plt.draw()
    os.makedirs(f"plots/qos/{direction}/{track_type}/{metric}", exist_ok=True)
    fig.savefig(f"plots/qos/{direction}/{track_type}/{metric}/connection_progression_{direction}_{track_type}_{metric}_{index_list_name[21:]}_{metric}.png", bbox_inches="tight")


In [ ]:
for i, index in enumerate(index_list_names):
    #draw_cons_plot_qos(i, index, "remote_inbound", "video", "roundTripTime", [0, 0.1, 0.15, 0.3, 0.5, float('inf')], ["darkblue", "green", "yellow", "orange", "black"])
        draw_cons_plot_qos(i, index, "remote_inbound", "video", "roundTripTime", 0, 0.5, ["darkblue", "green", "yellow", "orange", "black"])

In [ ]:
for i, index in enumerate(index_list_names):
    # draw_cons_plot_qos(i, index, "inbound", "video", "jitter", [0, 0.02, 0.03, 0.05, 0.1, float("inf")], ["darkblue", "green", "yellow", "orange", "black"])
    # draw_cons_plot_qos(i, index, "inbound", "audio", "jitter", [0, 0.02, 0.03, 0.05, 0.1, float("inf")], ["darkblue", "green", "yellow", "orange", "black"])
    draw_cons_plot_qos(i, index, "inbound", "video", "jitter", 0, 0.1, ["darkblue", "green", "yellow", "orange", "black"])
    draw_cons_plot_qos(i, index, "inbound", "audio", "jitter", 0, 0.1, ["darkblue", "green", "yellow", "orange", "black"])

In [ ]:
for i, index in enumerate(index_list_names):
    # draw_cons_plot_qos(i, index, "inbound", "video", "bitrate", [0, 500000, 800000, 1000000, 1500000, float('inf')], ["black", "orange", "yellow", "green", "darkblue"])
    # draw_cons_plot_qos(i, index, "inbound", "audio", "bitrate", [0, 10000, 16000, 32000, 64000, float('inf')], ["black", "orange", "yellow", "green", "darkblue"])
    # draw_cons_plot_qos(i, index, "outbound", "video", "bitrate", [0, 500000, 800000, 1000000, 1500000, float('inf')], ["black", "orange", "yellow", "green", "darkblue"])
    # draw_cons_plot_qos(i, index, "outbound", "audio", "bitrate", [0, 10000, 16000, 32000, 64000, float('inf')], ["black", "orange", "yellow", "green", "darkblue"])
    draw_cons_plot_qos(i, index, "inbound", "video", "bitrate", 0, 1500000, ["black", "orange", "yellow", "green", "darkblue"])
    draw_cons_plot_qos(i, index, "inbound", "audio", "bitrate", 0, 64000, ["black", "orange", "yellow", "green", "darkblue"])
    draw_cons_plot_qos(i, index, "outbound", "video", "bitrate", 0, 1500000, ["black", "orange", "yellow", "green", "darkblue"])
    draw_cons_plot_qos(i, index, "outbound", "audio", "bitrate", 0, 64000, ["black", "orange", "yellow", "green", "darkblue"])